In [19]:
import numpy as np
import pandas as pd

In [20]:
# retrieve the data
msolution_orders = pd.read_csv('MSolution Orders.csv')
nova_orders = pd.read_csv('nova orders.csv')
open_bars = pd.read_csv('order open bars.csv')

live_orders = pd.concat([msolution_orders, nova_orders])
open_bars

,Nova,datetime_ohlcv,MSolutions,datetime_ohlcv.1
0,"EURUSD,M2 (offline)",NaN,"EURUSD,M2 (offline)",NaN
1,Date,2024.03.11,Date,2024.03.11
2,Time,12:22,Time,12:01
3,Open,1.0942,Open,NaN
4,High,1.09435,High,NaN
...,...,...,...,...
363,Open,1.0924,NaN,NaN
364,High,1.0925,NaN,NaN
365,Low,1.09224,NaN,NaN
366,Close,1.0925,NaN,NaN


In [21]:
nova_open_bars = open_bars.loc[:, ['Nova', 'datetime_ohlcv']]
msolutions_open_bars = open_bars.loc[:, ['MSolutions', 'datetime_ohlcv.1']]
# drop nulls in the "MSolutions" column
msolutions_open_bars_no_na = msolutions_open_bars.dropna(subset=['MSolutions'])

In [22]:
def massaging_df(df, name_col, datetime_col):
    # remove any rows in name_col that contain the word "EURUSD"
    open_bars_no_symbol_label = df[~df[name_col].str.contains('EURUSD')]
    open_bars_datetime_Open = open_bars_no_symbol_label[open_bars_no_symbol_label[name_col].str.contains('Date') 
                                                        | open_bars_no_symbol_label[name_col].str.contains('Open') 
                                                        | open_bars_no_symbol_label[name_col].str.contains('Time')]
    # pivot the dataframe to be wide instead of long, so move all the unique values in the 'Nova' column to be their own columns
    pivot = open_bars_datetime_Open.pivot(columns=name_col, values=datetime_col)
    # forward fill all of the columns
    pivot_filled = pivot.ffill()
    # drop the rows with nulls
    pivot_filled_na_dropped = pivot_filled.dropna()
    # drop duplicates
    pivot_filled_na_dropped_no_dups = pivot_filled_na_dropped.drop_duplicates()
    # concatenate 'Date' and 'Time' columns
    pivot_filled_na_dropped_no_dups['Date'] = pivot_filled_na_dropped_no_dups['Date'].astype(str)
    pivot_filled_na_dropped_no_dups['Time'] = pivot_filled_na_dropped_no_dups['Time'].astype(str)
    pivot_filled_na_dropped_no_dups['datetime'] = pivot_filled_na_dropped_no_dups['Date'] + ' ' + pivot_filled_na_dropped_no_dups['Time']
    pivot_filled_na_dropped_no_dups['datetime'] = pd.to_datetime(pivot_filled_na_dropped_no_dups['datetime'], format='%Y.%m.%d %H:%M')
    # group by datetime, and get the second value of the index for 'Open' column
    pivot_filled_na_dropped_no_dups['Open'] = pivot_filled_na_dropped_no_dups['Open'].astype(float)
    # group by datetime, and get the count
    pivot_count = pivot_filled_na_dropped_no_dups.groupby('datetime')['Open'].count()
    # join pivot_count with pivot_filled_na_dropped_no_dups
    pivot_filled_na_dropped_no_dups_joined = pivot_filled_na_dropped_no_dups.join(pivot_count, on='datetime', rsuffix='_count', how='left')
    not_a_dup = pivot_filled_na_dropped_no_dups_joined[pivot_filled_na_dropped_no_dups_joined['Open_count'] < 2]
    with_dups = pivot_filled_na_dropped_no_dups_joined[pivot_filled_na_dropped_no_dups_joined['Open_count'] > 1]
    pivot_filled_na_dropped_no_dups_grouped = with_dups.groupby('datetime').nth(1)
    appended_df = pd.concat([not_a_dup, pivot_filled_na_dropped_no_dups_grouped])
    appended_df_sorted = appended_df.sort_values(by='datetime')

    return appended_df_sorted



In [23]:
import warnings
warnings.filterwarnings('ignore')
nova_df = massaging_df(nova_open_bars, 'Nova', 'datetime_ohlcv')
msolutions_df = massaging_df(msolutions_open_bars_no_na, 'MSolutions', 'datetime_ohlcv.1')
msolutions_df

,Date,Open,Time,datetime,Open_count
11,2024.03.11,1.0943,12:08,2024-03-11 12:08:00,1
19,2024.03.11,1.0937,13:11,2024-03-11 13:11:00,2
27,2024.03.11,1.0941,14:00,2024-03-11 14:00:00,2
35,2024.03.11,1.0942,14:32,2024-03-11 14:32:00,2
43,2024.03.11,1.0940,15:17,2024-03-11 15:17:00,2
51,2024.03.11,1.0936,15:51,2024-03-11 15:51:00,2
59,2024.03.11,1.0935,15:52,2024-03-11 15:52:00,2
67,2024.03.11,1.0927,16:32,2024-03-11 16:32:00,2
75,2024.03.11,1.0925,16:47,2024-03-11 16:47:00,2
83,2024.03.11,1.0920,17:15,2024-03-11 17:15:00,2


In [24]:
nova_df

,Date,Open,Time,datetime,Open_count
3,2024.03.11,1.0942,12:22,2024-03-11 12:22:00,1
11,2024.03.11,1.0936,13:07,2024-03-11 13:07:00,2
19,2024.03.11,1.0941,14:04,2024-03-11 14:04:00,2
27,2024.03.11,1.0942,14:32,2024-03-11 14:32:00,2
35,2024.03.11,1.0940,15:15,2024-03-11 15:15:00,2
43,2024.03.11,1.0936,15:51,2024-03-11 15:51:00,2
51,2024.03.11,1.0935,15:52,2024-03-11 15:52:00,2
59,2024.03.11,1.0927,16:32,2024-03-11 16:32:00,2
66,2024.03.11,1.0927,16:35,2024-03-11 16:35:00,1
74,2024.03.11,1.0927,16:36,2024-03-11 16:36:00,1


In [25]:
warnings.resetwarnings()

In [26]:
# convert Open Time to datetime 
msolution_orders['Open Time'] = pd.to_datetime(msolution_orders['Open Time'], format='%Y.%m.%d %H:%M:%S')
nova_orders['Open Time'] = pd.to_datetime(nova_orders['Open Time'], format='%Y.%m.%d %H:%M:%S')
# create a column that doesn't include the seconds for Open Time
msolution_orders['Open Time No Seconds'] = msolution_orders['Open Time'].dt.strftime('%Y.%m.%d %H:%M')
nova_orders['Open Time No Seconds'] = nova_orders['Open Time'].dt.strftime('%Y.%m.%d %H:%M')
# join msolution_orders and msolutions_df on 'Open Time No Seconds' for msolution_orders and 'datetime' for msolutions_df
msolutions_df['Open Time No Seconds'] = msolutions_df['datetime'].dt.strftime('%Y.%m.%d %H:%M')
nova_df['Open Time No Seconds'] = nova_df['datetime'].dt.strftime('%Y.%m.%d %H:%M')
# filter msolutions_df to only include the columns 'Open Time No Seconds' and 'Open'
msolutions_df_filtered = msolutions_df.loc[:, ['Open Time No Seconds', 'Open']]
nova_df_filtered = nova_df.loc[:, ['Open Time No Seconds', 'Open']]
msolutions_joined = msolution_orders.join(msolutions_df_filtered.set_index('Open Time No Seconds'), on='Open Time No Seconds', rsuffix='_msolutions', how='left')
nova_joined = nova_orders.join(nova_df_filtered.set_index('Open Time No Seconds'), on='Open Time No Seconds', rsuffix='_nova', how='left')

In [39]:
# find the difference between "Price" and "Open" for both msolutions_joined and nova_joined
# drop the null value from msolutions_joined in the Open column
msolutions_joined_no_na = msolutions_joined.dropna(subset=['Open'])
msolutions_joined_no_na['Spread'] = msolutions_joined_no_na['Price'] - msolutions_joined_no_na['Open']
nova_joined['Spread'] = nova_joined['Price'] - nova_joined['Open']
# create a column to calculate the close spread, which we can determine by shifting the "Open" column up by one row
msolutions_joined_no_na['Close'] = msolutions_joined_no_na['Open'].shift(-1)
nova_joined['Close'] = nova_joined['Open'].shift(-1)
# find the close spread
msolutions_joined_no_na['Close Spread'] = msolutions_joined_no_na['Close'] - msolutions_joined_no_na['Price.1']
nova_joined['Close Spread'] = nova_joined['Close'] - nova_joined['Price.1']
# filter to just the buy orders for the "Type" column in both dataframes
msolutions_joined_no_na_buy = msolutions_joined_no_na[msolutions_joined_no_na['Type'] == 'buy']
nova_joined_buy = nova_joined[nova_joined['Type'] == 'buy']
msolutions_joined_no_na_sell = msolutions_joined_no_na[msolutions_joined_no_na['Type'] == 'sell']
nova_joined_sell = nova_joined[nova_joined['Type'] == 'sell']

C:\Users\WilliamFetzner\AppData\Local\Temp\ipykernel_28164\519632357.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msolutions_joined_no_na['Spread'] = msolutions_joined_no_na['Price'] - msolutions_joined_no_na['Open']
C:\Users\WilliamFetzner\AppData\Local\Temp\ipykernel_28164\519632357.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msolutions_joined_no_na['Close'] = msolutions_joined_no_na['Open'].shift(-1)
C:\Users\WilliamFetzner\AppData\Local\Temp\ipykernel_28164\519632357.py:10: SettingWithCop

In [40]:
# find the average spread for both msolutions_joined and nova_joined
print(f"""MSolutions Average Spread Buy: {msolutions_joined_no_na_buy['Spread'].mean()}
       Nova Average Spread Buy: {nova_joined_buy['Spread'].mean()}
       MSolutions Average Spread Sell: {msolutions_joined_no_na_sell['Spread'].mean()}
       Nova Average Spread Sell: {nova_joined_sell['Spread'].mean()}
       MSolutions Average Close Spread Buy: {msolutions_joined_no_na_buy['Close Spread'].mean()}
       Nova Average Close Spread Buy: {nova_joined_buy['Close Spread'].mean()}
       MSolutions Average Close Spread Sell: {msolutions_joined_no_na_sell['Close Spread'].mean()}
       Nova Average Close Spread Sell: {nova_joined_sell['Close Spread'].mean()}""")

MSolutions Average Spread Buy: 0.00010045454545453742
       Nova Average Spread Buy: 8.521739130433652e-05
       MSolutions Average Spread Sell: -0.00012476190476188678
       Nova Average Spread Sell: -4.739130434782763e-05
       MSolutions Average Close Spread Buy: 0.00012190476190472521
       Nova Average Close Spread Buy: 0.00012454545454543275
       MSolutions Average Close Spread Sell: -0.00010904761904762501
       Nova Average Close Spread Sell: -0.00012782608695653856


In [41]:
# create a results dataframe from the above print statements with MSolutions and Nova as a column, and spread buy, spread sell, close spread buy, and close spread sell as rows
results = pd.DataFrame({'MSolutions': [msolutions_joined_no_na_buy['Spread'].mean(), msolutions_joined_no_na_sell['Spread'].mean(), 
                                       msolutions_joined_no_na_buy['Close Spread'].mean(), msolutions_joined_no_na_sell['Close Spread'].mean()],
                        'Nova': [nova_joined_buy['Spread'].mean(), nova_joined_sell['Spread'].mean(), nova_joined_buy['Close Spread'].mean(), 
                                 nova_joined_sell['Close Spread'].mean()]}, index=['Spread Buy', 'Spread Sell', 'Close Spread Buy', 'Close Spread Sell'])
results

,MSolutions,Nova
Spread Buy,0.000100,0.000085
Spread Sell,-0.000125,-0.000047
Close Spread Buy,0.000122,0.000125
Close Spread Sell,-0.000109,-0.000128


In [44]:
# find the range of dates for Open Time in msolutions_joined_no_na
start_time, end_time = msolutions_joined_no_na['Open Time'].min(), msolutions_joined_no_na['Open Time'].max()
# convert the start_time and end_time to a string that has the format 'MM_DD_YY'
start_time_str = start_time.strftime('%m_%d_%y')
end_time_str = end_time.strftime('%m_%d_%y')
# save the results to a csv
# results.to_csv(f'spread_results_{start_time_str}_{end_time_str}.csv')
print(f'spread_results_{start_time_str}_{end_time_str}.csv')

spread_results_03_11_24_03_12_24.csv


In [38]:
msolutions_joined_no_na

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price.1,Commission,Taxes,Swap,Profit,Open Time No Seconds,Open,Spread,Close,Close Spread
1,47707381,2024-03-11 12:08:33,sell,3,eurusd.i,1.09430,1.09459,0,2024.03.11 13:11:02,1.09382,-15,0,0,144,2024.03.11 12:08,1.0943,0.00000,1.0937,-0.00012
2,47711342,2024-03-11 13:11:03,buy,3,eurusd.i,1.09382,1.09354,0,2024.03.11 14:00:52,1.09388,-15,0,0,18,2024.03.11 13:11,1.0937,0.00012,1.0941,0.00022
3,47713783,2024-03-11 14:00:53,sell,3,eurusd.i,1.09389,1.09417,0,2024.03.11 14:03:18,1.09427,-15,0,0,-114,2024.03.11 14:00,1.0941,-0.00021,1.0942,-0.00007
4,47715182,2024-03-11 14:32:25,buy,3,eurusd.i,1.09429,1.09400,0,2024.03.11 15:17:11,1.09389,-15,0,0,-120,2024.03.11 14:32,1.0942,0.00009,1.0940,0.00011
5,47718057,2024-03-11 15:17:13,sell,3,eurusd.i,1.09388,1.09417,0,2024.03.11 15:20:06,1.09426,-15,0,0,-114,2024.03.11 15:17,1.0940,-0.00012,1.0936,-0.00066
6,47720747,2024-03-11 15:51:14,buy,3,eurusd.i,1.09365,1.09336,0,2024.03.11 15:52:02,1.09337,-15,0,0,-84,2024.03.11 15:51,1.0936,0.00005,1.0935,0.00013
7,47720804,2024-03-11 15:52:03,sell,3,eurusd.i,1.09337,1.09365,0,2024.03.11 16:32:14,1.09291,-15,0,0,138,2024.03.11 15:52,1.0935,-0.00013,1.0927,-0.00021
8,47725386,2024-03-11 16:32:15,buy,3,eurusd.i,1.09291,1.09262,0,2024.03.11 16:32:45,1.09253,-15,0,0,-114,2024.03.11 16:32,1.0927,0.00021,1.0925,-0.00003
9,47728308,2024-03-11 16:47:53,sell,3,eurusd.i,1.09238,1.09267,0,2024.03.11 17:15:35,1.09206,-15,0,0,96,2024.03.11 16:47,1.0925,-0.00012,1.0920,-0.00006
10,47733886,2024-03-11 17:15:35,buy,3,eurusd.i,1.09206,1.09178,0,2024.03.11 17:38:32,1.09180,-15,0,0,-78,2024.03.11 17:15,1.0920,0.00006,1.0919,0.00010


In [31]:
msolutions_joined_no_na.loc[:, ['Open Time', 'Type', 'Open', 'Price', 'Spread']]

,Open Time,Type,Open,Price,Spread
1,2024-03-11 12:08:33,sell,1.0943,1.09430,0.00000
2,2024-03-11 13:11:03,buy,1.0937,1.09382,0.00012
3,2024-03-11 14:00:53,sell,1.0941,1.09389,-0.00021
4,2024-03-11 14:32:25,buy,1.0942,1.09429,0.00009
5,2024-03-11 15:17:13,sell,1.0940,1.09388,-0.00012
6,2024-03-11 15:51:14,buy,1.0936,1.09365,0.00005
7,2024-03-11 15:52:03,sell,1.0935,1.09337,-0.00013
8,2024-03-11 16:32:15,buy,1.0927,1.09291,0.00021
9,2024-03-11 16:47:53,sell,1.0925,1.09238,-0.00012
10,2024-03-11 17:15:35,buy,1.0920,1.09206,0.00006
